### Récupération des données de statistiques d'Aftercinema

#### Vérification du bon fonctionnement de l'environnement + installations + imports

In [ ]:
print("hello")

In [ ]:
%pip install requests
%pip install pandas
%pip install sqlalchemy
%pip install psycopg2-binary
%pip install google-api-python-client
%pip install google-auth
%pip install google-auth-oauthlib
%pip install google-auth-httplib2
%pip install apify-client

In [ ]:
import os
import json
import requests
import pandas as pd
from sqlalchemy import create_engine, Table, Column, String, MetaData
from datetime import datetime,timedelta
import xml.etree.ElementTree as ET
from sqlalchemy.dialects.postgresql import JSONB
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from apify_client import ApifyClient

#### 1.Récupération des données PostHog

In [ ]:
token = os.getenv("POSTHOG_API_KEY")

headers = {"Authorization": "Bearer " + token}

with open("./POSTHOG_QUERIES.json", 'r') as fichier:
    queries = json.load(fichier)

events_data = {}

for event in ["Page viewed","Platform button clicked"]:
    data = {
        "query": {
            "kind": "HogQLQuery",
            "query": queries[event]
        }
    }

    request = requests.post("https://eu.posthog.com/api/projects/20861/query",headers=headers,json=data)

    data = request.json()

    df = pd.DataFrame(data["results"])
    df.columns = data["columns"]

    events_data[event] = df

Traitement des données pour le graphique PostHog - plateformes

In [ ]:
df_platforms = events_data["Platform button clicked"]

df_platforms['platform'] = df_platforms['platform'].replace({'Apple Podcasts': 'apple_podcasts','Apple Podcast': 'apple_podcasts', 'YouTube': 'youtube','Instagram':'instagram','Spotify':'spotify','Deezer':'deezer'})

df_platforms = df_platforms.groupby(['platform','person_device']).size().reset_index(name='count')

df_platforms.rename(columns={"person_device":"device"},inplace=True)
grouped = df_platforms.groupby('platform').agg({'count': 'sum'}).reset_index()
grouped['device'] = 'Tout appareil'
df_platforms = pd.concat([df_platforms, grouped], ignore_index=True)

platforms = df['platform'].unique()
devices = df_platforms['device'].unique()

df_platforms_full_index = pd.MultiIndex.from_product([platforms, devices], names=['platform', 'device'])
df_platforms_full = pd.DataFrame(index=df_platforms_full_index).reset_index()
df_platforms_full = pd.merge(df_platforms_full, df_platforms, on=['platform', 'device'], how='left').fillna(0)

posthog_result_platforms = {}
for category in devices:
    category_df_platforms = df_platforms_full[df_platforms_full['device'] == category]
    posthog_result_platforms[category] = [{'platform': row['platform'], 'count': int(row['count'])} for index, row in category_df_platforms.iterrows()]

Traitement des données pour le graphique PostHog - pages

In [ ]:
df_pages = events_data["Page viewed"]

df_pages['time'] = pd.to_datetime(df_pages['time'])
df_pages['year_month'] = df_pages['time'].dt.tz_localize(None).dt.to_period('M')

df_pages['page'] = df_pages['url'].apply(lambda x: 'listen' if '/listen' in x else ('stats' if '/stats' in x else 'home'))

df_pages['device'] = df_pages['person_device'].apply(lambda x: 'Mobile' if x == 'Tablet' else x)
df_pages = df_pages.groupby(['year_month', 'page','device']).size().reset_index(name='count')

grouped = df_pages.groupby(['year_month','page']).agg({'count': 'sum'}).reset_index()
grouped['device'] = 'Tout appareil'
df_pages = pd.concat([df_pages, grouped], ignore_index=True)

pages = df_pages['page'].unique()
devices = df_pages['device'].unique()
year_months = df_pages['year_month'].unique()

df_pages_full_index = pd.MultiIndex.from_product([pages, devices,year_months], names=['page', 'device','year_month'])
df_pages_full = pd.DataFrame(index=df_pages_full_index).reset_index()
df_pages_full = pd.merge(df_pages_full, df_pages, on=['page', 'device','year_month'], how='left').fillna(0)

posthog_result_pages = {}
for category in devices:
    category_df_pages = df_pages_full[df_pages_full['device'] == category].copy()
    category_df_pages.drop(columns=['device'], inplace=True)
    category_df_pages = category_df_pages.pivot(index='year_month', columns='page', values='count').fillna(0).reset_index()
    
    category_df_pages['home'] = category_df_pages['home'].astype(int)
    category_df_pages['listen'] = category_df_pages['listen'].astype(int)
    category_df_pages['stats'] = category_df_pages['stats'].astype(int)

    month_map = {1: 'jan', 2: 'fév', 3: 'mar', 4: 'avr', 5: 'mai', 6: 'jun',7: 'jui', 8: 'aoû', 9: 'sep', 10: 'oct', 11: 'nov', 12: 'déc'}
    category_df_pages['month'] = category_df_pages['year_month'].dt.month.map(month_map) + " " + (category_df_pages['year_month'].dt.year % 100).astype(str)
    category_df_pages.drop(columns=['year_month'], inplace=True)

    category_df_pages = category_df_pages[["month","home","listen","stats"]]

    posthog_result_pages[category]=category_df_pages.to_dict(orient='records')

#### 2. Récupération des données Acast

Récupération des titres et des ids des épisodes du podcast

In [ ]:
response = requests.get("https://feeds.acast.com/public/shows/aftercinema")
root = ET.fromstring(response.content)
namespaces = {'acast': 'https://schema.acast.com/1.0/'}
podcast_episodes = []

earliest_date = datetime.max

for item in root.findall('.//item'):
    episode_id = item.find('acast:episodeId', namespaces).text
    title = item.find('title').text
    pub_date = item.find('pubDate').text
    
    if datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %Z') < earliest_date:
        earliest_date = datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %Z')

    podcast_episodes.append({
        "id": episode_id,
        "aftercinema_id":"A"+episode_id,
        "title": title,
        "publishedDate": datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %Z').strftime('%Y-%m-%dT%H:%M:%S')
    })

In [ ]:
acast_token = os.getenv("ACAST_TOKEN")
headers = {"Authorization": "Bearer " + acast_token}
to_param = datetime.now()-timedelta(days=3)
params = {
    "from": "2024-02-19T23:00:00.000Z",
    "to": to_param,
    "interval": "day",
    "timeZone": "Europe/Paris"
}

Récupération des données de téléchargements

In [ ]:
url = "https://insights-api.acast.com/api/v2/charts/downloads/65d49906c4c0ce0016eadf8c/episode/"

full_df = pd.DataFrame()
for episode in podcast_episodes:
    response = requests.get(url+episode["id"], params=params,headers=headers)
    data = response.json()
    df = pd.DataFrame(data)
    df.rename(columns={'label': 'date'}, inplace=True)
    df['date'] = df['date'].str.split('T').str[0]
    df['title'] = episode["title"]
    full_df = pd.concat([full_df, df])

grouped = full_df.groupby('date').agg({'value': 'sum'}).reset_index()
grouped['title'] = 'Tous les épisodes'
acast_downloads_df = pd.concat([full_df, grouped], ignore_index=True)

Récupération des données des auditeurs

In [ ]:
url = "https://insights-api.acast.com/api/v2/shows/65d49906c4c0ce0016eadf8c/reach/histogram/episode/"

full_df = pd.DataFrame()
for episode in podcast_episodes:
    response = requests.get(url+episode["id"], params=params,headers=headers)
    data = response.json()
    df = pd.DataFrame(data)
    df.rename(columns={'label': 'date'}, inplace=True)
    df['date'] = df['date'].str.split('T').str[0]
    df['aftercinema_id'] = episode["aftercinema_id"]
    full_df = pd.concat([full_df, df])

grouped = full_df.groupby('date').agg({'value': 'sum'}).reset_index()
grouped['aftercinema_id'] = 'all_episodes'
full_df = pd.concat([full_df, grouped], ignore_index=True)

full_df['date'] = pd.to_datetime(full_df['date'])
full_df['date_only'] = full_df['date'].dt.date
filtered_df = full_df[full_df['date_only'] >= earliest_date.date()]
filtered_df = filtered_df.drop(columns=['date_only'])
filtered_df['date'] = filtered_df['date'].dt.strftime('%Y-%m-%d')

df_pivot = filtered_df.pivot(index='date', columns='aftercinema_id', values='value').reset_index()
result_listeners = df_pivot.to_dict(orient='records')

Récupération des données des plateformes

In [ ]:
url = "https://insights-api.acast.com/api/v2/shows/65d49906c4c0ce0016eadf8c/clients/histogram?clients=Spotify,Deezer,Apple+Podcasts,Other,Chrome,Safari,Acast+embed-player,Firefox,iVoox,CastBox,Podcast+Addict"
params["interval"] = "month"
response = requests.get(url, params=params,headers=headers)

result_platforms = []
autres_total_value = 0
specific_platforms = ['Spotify', 'Deezer', 'Apple Podcasts']

platforms_codes = {'Apple Podcasts': 'apple_podcasts','Spotify':'spotify','Deezer':'deezer'}

for platform in response.json():
    platform_name = platform['name']
    total_value = sum(item['value'] for item in platform['values'])
    
    if platform_name in specific_platforms:
        result_platforms.append({"platform": platforms_codes[platform_name], "value": total_value})
    else:
        autres_total_value += total_value

if autres_total_value > 0:
    result_platforms.append({"platform": "others", "value": autres_total_value})

#### 3. Récupération des données YouTube

Initialisation des requêtes aux APIs YouTube

In [ ]:
scopes = ['https://www.googleapis.com/auth/yt-analytics.readonly',"https://www.googleapis.com/auth/youtube.readonly"]

GOOGLE_AUTH_CLIENT_CONFIG = {
  "installed":{
    "client_id":os.getenv("GOOGLE_AUTH_CLIENT_ID"),
    "project_id":"aftercinema",
    "auth_uri":"https://accounts.google.com/o/oauth2/auth",
    "token_uri":"https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs",
    "client_secret":os.getenv("GOOGLE_AUTH_CLIENT_SECRET"),
    "redirect_uris":["http://localhost"]
  }
}

flow = InstalledAppFlow.from_client_config(GOOGLE_AUTH_CLIENT_CONFIG,scopes)
credentials = flow.run_local_server()
youtubeAnalytics = build("youtubeAnalytics", "v2", credentials = credentials)
youtube = build('youtube', 'v3', credentials=credentials)

Récupération des vues par jour et par vidéo

In [ ]:
video_list_response = youtube.playlistItems().list(
  part="snippet",
  playlistId="PLA75TyAwTPpmU0UuoPGXciOMcE9ipNLgT",
  maxResults=50
).execute()

youtube_views_df = pd.DataFrame()

video_list_response_filtered = [
    item for item in video_list_response['items']
    if item['snippet']['title'] != "Deleted video"
]

for video in video_list_response_filtered:
  video_id = video['snippet']['resourceId']['videoId']
  stats = youtubeAnalytics.reports().query(
    ids='channel==MINE',
    startDate='2024-02-19',
    endDate='2099-12-31',
    metrics='views',
    dimensions='day',
    sort='day',
    filters=f"video=={video_id}"
  ).execute()
  df = pd.DataFrame(stats['rows'], columns=['date', 'value'])
  df["title"]=video["snippet"]['title']
  youtube_views_df = pd.concat([youtube_views_df,df])

grouped = youtube_views_df.groupby('date').agg({'value': 'sum'}).reset_index()
grouped['title'] = 'Tous les épisodes'
youtube_views_df = pd.concat([youtube_views_df, grouped], ignore_index=True)

Récupération des informations globales YouTube et Acast

In [ ]:
dict_podcast_episodes = {item['title']: item for item in podcast_episodes}
general_episodes = []
for item1 in video_list_response['items']:
    normalized_title = item1['snippet']['title']
    if normalized_title in dict_podcast_episodes:
        item2 = dict_podcast_episodes[normalized_title]
        merged_dict = {
            'title': item2['title'],
            'acast_id': item2['id'],
            'aftercinema_id': item2['aftercinema_id'],
            'acast_publishedDate': item2['publishedDate'],
            'youtube_id': item1["snippet"]['resourceId']['videoId'],
            'youtube_publishedDate': datetime.strptime(item1['snippet']['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime('%Y-%m-%dT%H:%M:%S')
        }
        general_episodes.append(merged_dict)

Ajout des données YouTube aux données Acast (downloads)

In [ ]:
combined_df = pd.concat([youtube_views_df, acast_downloads_df])
acast_youtube_downloads = combined_df.groupby(['date', 'title']).agg({'value': 'sum'}).reset_index()

df_general_episodes = pd.DataFrame(general_episodes)
new_row = pd.DataFrame([{'title': 'Tous les épisodes', 'acast_id': None, 'aftercinema_id': 'all_episodes', 'acast_publishedDate': None, 'youtube_id': None, 'youtube_publishedDate': None}])
df_general_episodes = pd.concat([df_general_episodes, new_row], ignore_index=True)

df_acast_youtube_downloads = acast_youtube_downloads.merge(df_general_episodes[['title', 'aftercinema_id']], on='title', how='left')
df_acast_youtube_downloads = df_acast_youtube_downloads.drop(columns=['title'])

df_acast_youtube_downloads['date'] = pd.to_datetime(df_acast_youtube_downloads['date'])
df_acast_youtube_downloads['date_only'] = df_acast_youtube_downloads['date'].dt.date
df_acast_youtube_downloads_filtered_df = df_acast_youtube_downloads[df_acast_youtube_downloads['date_only'] >= earliest_date.date()]
df_acast_youtube_downloads_filtered_df = df_acast_youtube_downloads_filtered_df.drop(columns=['date_only'])
df_acast_youtube_downloads_filtered_df['date'] = df_acast_youtube_downloads_filtered_df['date'].dt.strftime('%Y-%m-%d')

df_pivot = df_acast_youtube_downloads_filtered_df.pivot(index='date', columns='aftercinema_id', values='value').reset_index()
result_downloads = df_pivot.to_dict(orient='records')

Ajout des données YouTube aux données Acast (platforms)

In [ ]:
stats = youtubeAnalytics.reports().query(
    ids='channel==MINE',
    startDate='2024-02-19',
    endDate='2099-12-31',
    metrics='views'
  ).execute()
result_platforms.append({"platform": "youtube", "value": stats["rows"][0][0]})

Récupération des données globale de YouTube

In [ ]:
youtubeAnalytics = build("youtubeAnalytics", "v2", credentials = credentials)
stats = youtubeAnalytics.reports().query(
    ids='channel==MINE',
    startDate='2024-02-19',
    endDate='2099-12-31',
    metrics='likes,subscribersGained,subscribersLost',
  ).execute()
youtube_result = {}
youtube_result["likesCount"] = stats["rows"][0][0]
youtube_result["subscribersCount"] = stats["rows"][0][1]-stats["rows"][0][2]

#### Récupération des données Instagram

In [ ]:
client = ApifyClient(os.getenv("APIFY_TOKEN"))
run = client.actor("dSCLg0C3YEZ83HzYX").call(run_input={ "usernames": ["aftercinema.podcast"] })
apify_result = client.dataset(run["defaultDatasetId"]).list_items().items[0]
instagram_result = {}
instagram_result["followersCount"] = apify_result["followersCount"]
instagram_result["postsCount"] = apify_result["postsCount"]
instagram_result["likesCount"] = 0
for post in apify_result["latestPosts"]:
    instagram_result["likesCount"] = instagram_result["likesCount"] + post["likesCount"]

#### Stockage du résultat

In [ ]:
date = datetime.now().strftime("%Y-%m-%d %H:%M")

def convert_data(data):
    return json.dumps(data, ensure_ascii=False)

data_to_insert = [
    {
        "data_name":"PostHog - Page viewed",
        "data":posthog_result_pages,
        "date": date
    },
    {
        "data_name":"PostHog - Platform button clicked",
        "data":posthog_result_platforms,
        "date": date
    },
    {
        "data_name":"Acast+Youtube - Downloads",
        "data":result_downloads,
        "date": date
    },
    {
        "data_name":"Acast - Listeners",
        "data":result_listeners,
        "date": date
    },
    {
        "data_name":"Acast+Youtube - Platforms",
        "data":result_platforms,
        "date": date
    },
    {
        "data_name":"Instagram",
        "data":instagram_result,
        "date": date
    },
    {
        "data_name":"General - Episodes",
        "data":sorted(general_episodes, key=lambda x: x['acast_publishedDate']),
        "date": date
    },
    {
        "data_name":"YouTube",
        "data":youtube_result,
        "date": date
    }
]

engine = create_engine(os.getenv("POSTGRESQL_CONN_STRING"))

metadata = MetaData()
table = Table('stats_data', metadata,
              Column('data_name', String),
              Column('data', JSONB),
              Column('date', String))

with engine.connect() as connection:
    with connection.begin() as transaction:
        table.drop(engine, checkfirst=True)
        transaction.commit()

metadata.create_all(engine)
for line in data_to_insert:
    with engine.connect() as connection:
        with connection.begin() as transaction:
            connection.execute(table.insert().values(data_name=line["data_name"],data=line["data"],date=line["date"]))
            transaction.commit()